In [ ]:
# Login to Hugging Face
from huggingface_hub import login
from google.colab import userdata
hf_token = userdata.get('HugFACE')  # Replace with your token if necessary
login(token=hf_token)

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

teacher_model_name = "deepset/xlm-roberta-large-squad2"
teacher_model = AutoModelForQuestionAnswering.from_pretrained(teacher_model_name)
teacher_tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
print(sum(p.numel() for p in teacher_model.parameters()))

558842882


In [ ]:
nlp_1 = pipeline('question-answering', model=teacher_model_name, tokenizer=teacher_model_name)
QA_input_1 = {
    'question': 'இந்தியாவில் பேசப்படும் மொழி என்ன?',
    'context': 'இந்தியாவில் அதிக மொழிகள் பேசப்படுகின்றன. அங்கு இந்தி, தமிழ், கன்னடம், மலையாளம் போன்றவை உள்ளன.'
}
res_1 = nlp_1(QA_input_1)
print(res_1)

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  wa

{'score': 0.8051934838294983, 'start': 46, 'end': 53, 'answer': ' இந்தி,'}


In [ ]:
student_model_name = "Khanh/distilbert-base-multilingual-cased-finetuned-squad"
student_model= AutoModelForQuestionAnswering.from_pretrained(student_model_name)
student_tokenizer= AutoTokenizer.from_pretrained(student_model_name)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
print(sum(p.numel() for p in student_model.parameters()))

134735618


In [ ]:
nlp_2= pipeline('question-answering', model=student_model, tokenizer=student_tokenizer)
res_2 = nlp_2(QA_input_1)
print(res_2)

Device set to use cpu


{'score': 0.2869168817996979, 'start': 12, 'end': 39, 'answer': 'அதிக மொழிகள் பேசப்படுகின்றன'}


In [ ]:
student_model.resize_token_embeddings(len(teacher_tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(250002, 768, padding_idx=0)

In [ ]:
if teacher_tokenizer.pad_token is None:
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token

In [ ]:
# Update the model configurations with the padding token
teacher_model.config.pad_token_id = teacher_tokenizer.pad_token_id
student_model.config.pad_token_id = teacher_tokenizer.pad_token_id

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("RajeevanL/tamil_squad-2.0")
print(ds)

README.md:   0%|          | 0.00/573 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.15M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.30M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/66277 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5848 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5848 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Context', 'Answer'],
        num_rows: 66277
    })
    validation: Dataset({
        features: ['Question', 'Context', 'Answer'],
        num_rows: 5848
    })
    test: Dataset({
        features: ['Question', 'Context', 'Answer'],
        num_rows: 5848
    })
})


In [ ]:
train_dataset = ds['train']
validation_dataset = ds['validation']
test_dataset = ds['test']

In [ ]:
# Define the customized prompt for your dataset
custom_prompt = """Below is a question that needs an answer, paired with context that provides additional information. Write a response that appropriately answers the question.

### Question:
{}

### Context:
{}

### Answer:
{}"""
# Define the EOS token (adjust based on your tokenizer)
EOS_TOKEN = "<|endoftext|>"  # Replace with your tokenizer's actual EOS token

# Formatting function for the dataset
def formatting_prompts_func(examples):
    # Extract fields from the dataset
    questions = examples["Question"]
    contexts = examples["Context"]
    answers = examples["Answer"]

    texts = []
    for question, context, answer in zip(questions, contexts, answers):
        # Format the text with the Alpaca-style prompt
        text = custom_prompt.format(question, context, answer) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

In [ ]:
# Apply formatting to the DatasetDict
print("Formatting Dataset...")
formatted_dataset = ds.map(formatting_prompts_func, batched=True)

# Verify the output
print(formatted_dataset)

Formatting Dataset...


Map:   0%|          | 0/66277 [00:00<?, ? examples/s]

Map:   0%|          | 0/5848 [00:00<?, ? examples/s]

Map:   0%|          | 0/5848 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Question', 'Context', 'Answer', 'text'],
        num_rows: 66277
    })
    validation: Dataset({
        features: ['Question', 'Context', 'Answer', 'text'],
        num_rows: 5848
    })
    test: Dataset({
        features: ['Question', 'Context', 'Answer', 'text'],
        num_rows: 5848
    })
})


In [ ]:
print(student_tokenizer.model_input_names)

['input_ids', 'attention_mask']


In [ ]:
def tokenize_function(examples):
    return student_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

In [ ]:
# Apply tokenization to the new dataset
tokenized_dataset = formatted_dataset['train'].map(tokenize_function, batched=True, remove_columns=['Question', 'Context', 'Answer'])

# Prepare dataset for distillation
train_data = tokenized_dataset
print(train_data)

# Data collator for language modeling
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=student_tokenizer,
    mlm=False  # Set to False for causal language modeling
)

Map:   0%|          | 0/66277 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 66277
})


In [ ]:
# Ensure the dataset has 'input_ids' and 'attention_mask'
if 'input_ids' not in tokenized_dataset.column_names or 'attention_mask' not in tokenized_dataset.column_names:
    raise ValueError("The tokenized dataset must contain 'input_ids' and 'attention_mask'.")

#New Loss Function
We will subclass Trainer and overriding the compute_loss() method to include the knowledge distillation loss term LKD:



In [ ]:
!pip install transformers
from transformers import TrainingArguments

class KnowledgeDistillationTrainingArguments(TrainingArguments):
  def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
    super().__init__(*args, **kwargs)
    self.alpha = alpha
    self.temperature = temperature

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer
import torch
device = torch.device("cpu")

In [ ]:
import torch
import torch.nn.functional as F
from transformers import Trainer


class KnowledgeDistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model.to(device)
        self.model.to(device)

        # Set up the optimizer for the student model
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None, **kwargs):
        # Move inputs to the device
        inputs = {key: value.to(device) for key, value in inputs.items()}

        if hasattr(model, 'embeddings') and model.embeddings is not None:
            model.embeddings.to(device)

        if hasattr(self.teacher_model, 'embeddings') and self.teacher_model.embeddings is not None:
            self.teacher_model.embeddings.to(device)

        # Student outputs
        outputs_student = model(**inputs)
        start_logits_student = outputs_student.start_logits
        end_logits_student = outputs_student.end_logits

        # Teacher outputs
        with torch.no_grad():
            outputs_teacher = self.teacher_model(**inputs)
            start_logits_teacher = outputs_teacher.start_logits
            end_logits_teacher = outputs_teacher.end_logits

        # KL divergence loss
        loss_fct = torch.nn.KLDivLoss(reduction="batchmean")
        loss_kd_start = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(start_logits_student / self.args.temperature, dim=-1),
            F.softmax(start_logits_teacher / self.args.temperature, dim=-1)
        )
        loss_kd_end = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(end_logits_student / self.args.temperature, dim=-1),
            F.softmax(end_logits_teacher / self.args.temperature, dim=-1)
        )

        # Final loss
        loss_ce = outputs_student.loss
        loss_kd = loss_kd_start + loss_kd_end
        loss = self.args.alpha * loss_ce + (1.0 - self.args.alpha) * loss_kd

        return (loss, outputs_student) if return_outputs else loss


#Metrics for DistillationTrainer

In [ ]:
!pip install evaluate
!pip install transformers[torch]
!pip install accelerate>=0.20.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:00


In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
from transformers import EvalPrediction

# Define the compute_metrics function
def compute_metrics(pred):
    try:
        # Unpack predictions and labels
        start_logits, end_logits = pred.predictions
        labels = pred.label_ids

        # Ensure labels are a NumPy array
        labels = np.array(labels)

        # Debugging: Log shapes
        print(f"Start logits shape: {start_logits.shape}")
        print(f"End logits shape: {end_logits.shape}")
        print(f"Labels shape (before processing): {labels.shape}")

        # Reshape or transpose labels if necessary
        if labels.ndim == 2 and labels.shape[0] == 2:
            labels = labels.T  # Transpose to (n_samples, 2)

        # Extract only the first two columns if labels have extra dimensions
        if labels.ndim == 2 and labels.shape[1] > 2:
            labels = labels[:, :2]

        # Validate labels shape
        if labels.ndim != 2 or labels.shape[1] != 2:
            raise ValueError(
                f"Labels should have shape (n_samples, 2), but found {labels.shape}. "
                f"Ensure your dataset returns [start_idx, end_idx]."
            )

        # Convert logits to predicted indices
        start_preds = np.argmax(start_logits, axis=-1)
        end_preds = np.argmax(end_logits, axis=-1)

        # Extract true start and end indices
        true_start = labels[:, 0]
        true_end = labels[:, 1]

        # Compute token-level accuracies
        start_accuracy = accuracy_score(true_start, start_preds)
        end_accuracy = accuracy_score(true_end, end_preds) if np.any(true_end != -1) else None
        avg_accuracy = (start_accuracy + (end_accuracy or 0)) / (2 if end_accuracy is not None else 1)

        return {
            "start_accuracy": start_accuracy,
            "end_accuracy": end_accuracy,
            "avg_accuracy": avg_accuracy
        }

    except Exception as e:
        raise ValueError(f"Error in compute_metrics: {e}")


# Training Arguments for DistillationTrainer

In [ ]:
batch_size = 64
finetuned_student_ckpt = "finetuned-tamil_Roberta"

In [ ]:
student_training_args = KnowledgeDistillationTrainingArguments(
    output_dir=finetuned_student_ckpt,
    eval_strategy = "epoch",
    num_train_epochs=1,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    alpha=1,
    weight_decay=0.01
    )

## Lets initialize student model but before that provide the student model with the mappings between each intent and label ID.

In [ ]:
from transformers import AutoConfig

# Load the configuration for the QA model
student_config = AutoConfig.from_pretrained(student_model_name)
print(student_config)

DistilBertConfig {
  "_name_or_path": "Khanh/distilbert-base-multilingual-cased-finetuned-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.47.1",
  "vocab_size": 119547
}



In [ ]:
import torch
from transformers import AutoModelForQuestionAnswering

# Function to initialize the student model for Question Answering
def student_init():
    return (
        AutoModelForQuestionAnswering
        .from_pretrained(student_model_name, config=student_config)
        .to(device)
    )


## Lets Load teacher checkpoint and start finetuning

In [ ]:
teacher_config = AutoConfig.from_pretrained(teacher_model_name)
print(teacher_config)

XLMRobertaConfig {
  "_name_or_path": "deepset/xlm-roberta-large-squad2",
  "architectures": [
    "XLMRobertaForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "language": "english",
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "name": "XLMRoberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.47.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}



In [ ]:
!pip install wandb

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "false"
# Resize the student model’s token embeddings to match the teacher model's vocabulary size
student_model.resize_token_embeddings(len(teacher_tokenizer))

Embedding(250002, 768, padding_idx=0)

In [ ]:
# Set the padding token for both teacher and student tokenizers
if teacher_tokenizer.pad_token is None:
    teacher_tokenizer.pad_token = teacher_tokenizer.eos_token

if student_tokenizer.pad_token is None:
    student_tokenizer.pad_token = student_tokenizer.eos_token

In [ ]:
from datasets import DatasetDict
from transformers import AutoTokenizer

def preprocess_function(examples):
    """
    Preprocess the dataset for question-answering by tokenizing questions and contexts,
    and mapping answer spans to token indices.
    """
    # Extract questions, contexts, and answers
    questions = examples["Question"]
    contexts = examples["Context"]
    answers = examples["Answer"]

    # Ensure all inputs are strings (handle missing or invalid inputs)
    questions = [str(q) if q is not None else "" for q in questions]
    contexts = [str(c) if c is not None else "" for c in contexts]
    answers = [str(a) if a is not None else "" for a in answers]

    # Tokenize questions and contexts
    encodings = student_tokenizer(
        questions,
        contexts,
        truncation=True,
        padding=True,
        max_length=512
    )

    # Calculate start and end positions for answers
    start_positions = []
    end_positions = []

    for i in range(len(answers)):
        answer = answers[i]
        context = contexts[i]

        # If context or answer is empty, skip
        if not context or not answer:
            start_positions.append(0)
            end_positions.append(0)
            continue

        # Find the start and end character positions of the answer in the context
        start_char = context.find(answer)
        end_char = start_char + len(answer)

        # Map character positions to token indices
        tokenized_context = teacher_tokenizer(
            context,
            truncation=True,
            max_length=512,
            return_offsets_mapping=True,
            add_special_tokens=False
        )
        offsets = tokenized_context["offset_mapping"]

        # Initialize default positions
        start_token_idx = 0
        end_token_idx = 0

        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token_idx = idx
            if start < end_char <= end:
                end_token_idx = idx
                break

        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

    encodings["start_positions"] = start_positions
    encodings["end_positions"] = end_positions

    return encodings


# Apply preprocessing to the train, validation, and test splits
processed_datasets = DatasetDict({
    split: dataset.map(preprocess_function, batched=True, batch_size=8, remove_columns=["Question", "Context", "Answer", "text"])
    for split, dataset in formatted_dataset.items()
})

# Check the processed dataset
print(processed_datasets)


Map:   0%|          | 0/66277 [00:00<?, ? examples/s]

Map:   0%|          | 0/5848 [00:00<?, ? examples/s]

Map:   0%|          | 0/5848 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 66277
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5848
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
        num_rows: 5848
    })
})


In [ ]:
teacher_model.to(device)
student_model.to(device)

DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [ ]:
# Limit training and testing datasets to the required sizes
train_subset = processed_datasets["train"].select(range(500))  # Take the first 1000 samples for training
test_subset = processed_datasets["test"].select(range(50))    # Take the first 100 samples for testing


In [ ]:
print(train_subset)

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 500
})


In [ ]:
# Initialize the trainer with the subsets
distilbert_trainer = KnowledgeDistillationTrainer(
    model_init=student_init,  # Initialize the student model
    teacher_model=teacher_model,  # Teacher model for distillation
    args=student_training_args,  # Training arguments
    train_dataset=train_subset,  # Subset of training data
    eval_dataset=test_subset,    # Subset of testing data
    # compute_metrics=compute_metrics,  # Function to calculate evaluation metrics
    tokenizer=student_tokenizer,  # Tokenizer for the student model
)

# Start training
distilbert_trainer.train()

<ipython-input-41-1d4d6afde7c3>:8: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `KnowledgeDistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,2.835642


TrainOutput(global_step=8, training_loss=3.668705701828003, metrics={'train_runtime': 1249.764, 'train_samples_per_second': 0.4, 'train_steps_per_second': 0.006, 'total_flos': 65326550016000.0, 'train_loss': 3.668705701828003, 'epoch': 1.0})

In [ ]:
# Post-training evaluation: Compute metrics
# Get predictions from the trainer
predictions = distilbert_trainer.predict(test_dataset=test_subset)

# Create an EvalPrediction object
eval_prediction = EvalPrediction(predictions=predictions.predictions, label_ids=predictions.label_ids)


In [ ]:
# Compute and display metrics
metrics = compute_metrics(eval_prediction)
print(f"Post-Distillation Evaluation Metrics:\n{metrics}")

Start logits shape: (50, 512)
End logits shape: (50, 512)
Labels shape (before processing): (2, 50)
Post-Distillation Evaluation Metrics:
{'start_accuracy': 0.98, 'end_accuracy': 0.1, 'avg_accuracy': 0.54}


In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
finetuned_student= "/content/finetuned-tamil_Roberta/checkpoint-8"
# Reload the model and tokenizer from the saved directory
model = AutoModelForQuestionAnswering.from_pretrained(finetuned_student)
tokenizer = AutoTokenizer.from_pretrained(finetuned_student)

# Push to the Hugging Face Hub
model.push_to_hub("RajeevanL/xlm_roberta_distilled_v_1")
tokenizer.push_to_hub("RajeevanL/xlm_roberta_distilled_v_1")

model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RajeevanL/xlm_roberta_distilled_v_1/commit/f085cc735761e562d60528ded0cd019757648b93', commit_message='Upload tokenizer', commit_description='', oid='f085cc735761e562d60528ded0cd019757648b93', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RajeevanL/xlm_roberta_distilled_v_1', endpoint='https://huggingface.co', repo_type='model', repo_id='RajeevanL/xlm_roberta_distilled_v_1'), pr_revision=None, pr_num=None)

In [ ]:
distill_dir = "RajeevanL/xlm_roberta_distilled_v_1"

In [ ]:
distill_model = AutoModelForQuestionAnswering.from_pretrained(distill_dir)
distill_tokenizer = AutoTokenizer.from_pretrained(distill_dir)

model.safetensors:   0%|          | 0.00/539M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [ ]:
print(sum(p.numel() for p in distill_model.parameters()))

134735618


In [ ]:
qa_pipeline = pipeline("question-answering", model=distill_model, tokenizer=distill_tokenizer)

Device set to use cpu


In [ ]:
question =  'இந்தியாவில் பேசப்படும் மொழி என்ன?',
context = 'இந்தியாவில் அதிக மொழிகள் பேசப்படுகின்றன. அங்கு இந்தி, தமிழ், கன்னடம், மலையாளம் போன்றவை உள்ளன.'

In [ ]:
# Get the answer
result = qa_pipeline(QA_input_1)

# Print the result
print(result)

{'score': 0.00109675747808069, 'start': 0, 'end': 39, 'answer': 'இந்தியாவில் அதிக மொழிகள் பேசப்படுகின்றன'}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


In [ ]:
# Initialize Trainer for Teacher Model
teacher_training_args = TrainingArguments(
    output_dir="./results_teacher",
    evaluation_strategy="epoch",  # Adjust the strategy as necessary
)
teacher_trainer = Trainer(
    model=teacher_model,
    args=teacher_training_args,
    tokenizer=teacher_tokenizer,
    eval_dataset=test_subset,
    # Add other necessary parameters (like train_dataset, eval_dataset, etc.)
)


<ipython-input-79-4eaa08414579>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  teacher_trainer = Trainer(


In [ ]:
# Initialize Trainer for Student Model
student_training_args = TrainingArguments(
    output_dir="./results_student",
    evaluation_strategy="epoch",  # Adjust the strategy as necessary
)
student_trainer = Trainer(
    model=student_model,
    args=student_training_args,
    tokenizer=student_tokenizer,
     eval_dataset=test_subset,
    # Add other necessary parameters (like train_dataset, eval_dataset, etc.)
)

<ipython-input-81-fe84fd41de61>:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  student_trainer = Trainer(


In [ ]:
# Assuming you have both teacher_trainer and student_trainer set up

# Get predictions for the Teacher Model
teacher_predictions = teacher_trainer.predict(test_dataset=test_subset)
teacher_eval_prediction = EvalPrediction(predictions=teacher_predictions.predictions, label_ids=teacher_predictions.label_ids)

# Get predictions for the Student Model
student_predictions = student_trainer.predict(test_dataset=test_subset)
student_eval_prediction = EvalPrediction(predictions=student_predictions.predictions, label_ids=student_predictions.label_ids)

# Compute and print accuracy for Teacher Model
teacher_metrics = compute_metrics(teacher_eval_prediction)
print("Teacher Model Accuracy:", teacher_metrics)

# Compute and print accuracy for Student Model
student_metrics = compute_metrics(student_eval_prediction)
print("Student Model Accuracy:", student_metrics)


Start logits shape: (50, 512)
End logits shape: (50, 512)
Labels shape (before processing): (2, 50)
Teacher Model Accuracy: {'start_accuracy': 0.0, 'end_accuracy': 0.02, 'avg_accuracy': 0.01}
Start logits shape: (50, 512)
End logits shape: (50, 512)
Labels shape (before processing): (2, 50)
Student Model Accuracy: {'start_accuracy': 0.0, 'end_accuracy': 0.0, 'avg_accuracy': 0.0}


## Lets compare Teacher and Student Model

In [ ]:
#We will compare the two models based on size and inference time

Saving Teacher and Student model and then computing model's size in MB

In [ ]:
def save_teacher_model():
  teacher_model.save_pretrained("teacher_model")
def save_student_model():
  distilbert_trainer.save_model('student_model')


In [ ]:
save_teacher_model()
save_student_model()

In [ ]:
from transformers import AutoConfig, AutoModelForQuestionAnswering
import os

def compute_parameters(model_path):
  model = AutoModelForQuestionAnswering.from_pretrained(model_path)
  parameters = model.num_parameters()
  return parameters

In [ ]:
teacher_model_parameters = compute_parameters(model_path="/content/teacher_model")
print("Teacher Model: ", teacher_model_parameters)

In [ ]:
student_model_parameters = compute_parameters(model_path="/content/student_model")
print("Student Model: ", student_model_parameters)

In [ ]:
nlp_3= pipeline('question-answering', model=student_model, tokenizer=student_model_name)
QA_input_2 = {
    'question': 'திருக்குறளை எழுதியது யார்?',
    'context': 'திருக்குறள் என்பது தமிழின் புகழ்பெற்ற நூல்களில் ஒன்றாகும். இது திருவள்ளுவர் எனும் முனிவரால் எழுதப்பட்டது. இதில் 1330 குறள்கள் அடங்கியுள்ளன.'
}
res_3 = nlp_2(QA_input_2)
print(res_3)

In [ ]:
# from transformers import AutoModelForQuestionAnswering, AutoTokenizer
# model_path="/content/student_model"
# # Reload the model and tokenizer from the saved directory
# model = AutoModelForQuestionAnswering.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# # Push to the Hugging Face Hub
# model.push_to_hub("vibulan73/xlm_roberta_distilled_v1")
# tokenizer.push_to_hub("vibulan73/xlm_roberta_distilled_v1")

In [ ]:
decrease = (student_model_parameters-teacher_model_parameters)/teacher_model_parameters
print(decrease*100)

In [ ]:
!ls /content/student_model -al --block-size=MB

In [ ]:
!ls /content/teacher_model -al --block-size=MB

In [ ]:
print(formatted_dataset['train'][0])

In [ ]:
dataset = formatted_dataset['train'].map(lambda example: {'instruction_input': example['instruction'] + " " + example['input']},
                      remove_columns=['instruction','input'])

In [ ]:
print(dataset[0])

In [ ]:
#we will take average times of multiple inferences on same input

In [ ]:
# #Lets warmup first
# from transformers import pipeline
# import time

# pipe = pipeline("text-classification", model="/content/teacher_model", tokenizer='bert-base-uncased')

# sample_input = clinc['train']['text'][101]

# #WARMUP
# for _ in range(10):
#   _ = pipe(sample_input)

# start = time.time()
# for _ in range(100):
#   _ = pipe(sample_input)
# total_time_teacher_model = time.time()-start
# print("Total time to process 100 requests for Teacher Model: ",total_time_teacher_model)

In [ ]:
# pipe = pipeline("text-classification", model="/content/student_model", tokenizer="distilbert-base-uncased")

# sample_input = clinc['train']['text'][101]

# #WARMUP
# for _ in range(10):
#   _ = pipe(sample_input)

# start = time.time()
# for _ in range(100):
#   _ = pipe(sample_input)
# total_time_student_model = time.time()-start

# print("Total time to process 100 requests for Student Model: ",total_time_student_model)

In [ ]:
# decrease_in_time = (total_time_teacher_model-total_time_student_model)/total_time_teacher_model
# print(decrease_in_time*100)